In [35]:
import os
import csv
import requests
import urllib.parse
import pandas as pd
import IPython.display as Disp
from PIL import Image, ImageEnhance
from colors import color
from numpy import interp, random
import logging

from io import BytesIO

# Book Cover Processing

In [2]:
SEARCH_URL = "https://openlibrary.org/search.json{}"
COVER_URL = "https://covers.openlibrary.org/b/isbn/{}-L.jpg"

class BookNotFoundException(Exception):
    pass

class CoverNotFoundException(Exception):
    pass

class HTTPCodeException(Exception):
    pass

In [3]:
logger = logging.getLogger('my_logger')
logger.setLevel(logging.DEBUG)

In [4]:
# load data
data_file = os.path.join('../data/banned_books_clean.pickle')
data = pd.read_pickle(data_file)

In [5]:
data.loc[data["title"].str.contains("anus")]

,ID,author,title,challenge_type,challenger,decision,library_type,day,month,year,challenge_date,state


In [6]:
def get_book_info_and_cover(author, title):
    # set up logger       
    log_format = f"%(asctime)s - %(levelname)s - Author: {author} - Book: {title} - Message: %(message)s"
    formatter = logging.Formatter(log_format)
    handler = logging.StreamHandler()
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    
    # sanatize author and title
    safe_author = urllib.parse.quote_plus(author)
    safe_title = urllib.parse.quote_plus(title)

    # build the query url for the search
    query = f"?author={safe_author}&q={safe_title}"
    query_url = SEARCH_URL.format(query)

    # execute request for search url
    logger.info(f"fetching: '{query_url}'")
    response = requests.get(query_url)

    # raise if there is a non success code
    if response.status_code != 200:
        raise HTTPCodeException(f"url: {query_url} returned {response.status_code}")
        
    matching_books = response.json()

    # if there are no matches
    if matching_books["numFound"] == 0:
        raise BookNotFoundException(f"Could not find and matches for author: '{author}', title: '{title}'")

    first_book = matching_books["docs"][0]
    for isbn in first_book["isbn"]: 
        query_url = COVER_URL.format(isbn)
        
        logger.info(f"fetching: '{query_url}'")
        cover = requests.get(query_url)
        cover_image = Image.open(BytesIO(cover.content))
        
        if cover_image.height > 1 and cover_image.width > 1:
            break
        else:
            logger.warning(f"Cover not located for isbn: '{isbn}'")
    
    if cover_image.height == 1 and cover_image.width == 1:
        raise CoverNotFoundException(f"No covers found for any isbn in: {','.join(first_book['isbn'])}")

    logger.info(f"All info located")
    return {
        "author": author,
        "title": title,
        "cover": Image.open(BytesIO(cover.content))
    }

In [25]:
def render_ascii(img, new_width=80):
    chars =  " .,-~:;!=*#$@"
    
    _, _, width, height = img.getbbox()

    new_height = int((width / height) * new_width)
    
    img = img.resize((new_width, int(new_height)))
    pixels = img.getdata()

    new_pixels = []
    for pixel in pixels:
        luminance = .2126 * pixel[0] + .7152 * pixel[1] + .0722 * pixel[2]
        # clip the pixel from 255 values to the number of possible values in the chars string
        new_pixels.append(chars[int(interp(luminance, [0, 256], [0, len(chars)]))])

    colors = pixels_as_hex(pixels)
    
    new_pixels_count = len(new_pixels)
    ascii_picture = [new_pixels[index:index+new_width]
                     for index in range(0, new_pixels_count, new_width)]
    return ascii_picture, colors

In [26]:
def pixels_as_hex(pixels):
    return [rgb_to_hex(pixel[0], pixel[1], pixel[2]) for pixel in pixels]    

In [27]:
def rgb_to_hex(r, g, b):
    def _base_sixteen(num, numerals="0123456789ABCDEF"):
        if num < len(numerals):
            return numerals[num]
        else:
            return numerals[num // 16] + _base_sixteen(num % 16)

    r = _base_sixteen(r)
    g = _base_sixteen(g)
    b = _base_sixteen(b)
    return f"#{r:02}{g:02}{b:02}"

In [38]:
# Get the top 10 banned books in US public libraries
top_banned_books = data \
    .groupby('title')['ID'] \
    .nunique() \
    .sort_values(ascending=False)

# display(top_banned_books)
searches = []
for name, _ in top_banned_books.items():
    book = data.loc[data['title'] == name].iloc[0]
    searches.append((book.author, book.title))

In [58]:
# rip top 10 covers
all_info = []
# print(searches)

idx = int(random.default_rng().uniform(low=0, high=len(searches)))
print(idx)
author, title = searches[idx:idx + 1][0]
try:
    book_info = get_book_info_and_cover(author, title)
    all_info.append(book_info)
except Exception as e:
    logger.exception(str(e))

2023-11-08 10:19:24,093 - INFO - Author: Kobabe, Maia - Book: Gender Queer: A Memoir - Message: fetching: 'https://openlibrary.org/search.json?author=Lucas%2C+Chad&q=Thanks+a+Lot%2C+Universe'
2023-11-08 10:19:24,093 - INFO - Author: Kobabe, Maia - Book: Gender Queer: A Memoir - Message: fetching: 'https://openlibrary.org/search.json?author=Lucas%2C+Chad&q=Thanks+a+Lot%2C+Universe'
2023-11-08 10:19:24,093 - INFO - Author: Rud, Jeff - Book: Crossover - Message: fetching: 'https://openlibrary.org/search.json?author=Lucas%2C+Chad&q=Thanks+a+Lot%2C+Universe'
2023-11-08 10:19:24,093 - INFO - Author: Rocha, Lucas - Book: Where We Go From Here - Message: fetching: 'https://openlibrary.org/search.json?author=Lucas%2C+Chad&q=Thanks+a+Lot%2C+Universe'
2023-11-08 10:19:24,093 - INFO - Author: Lucas, Chad - Book: Thanks a Lot, Universe - Message: fetching: 'https://openlibrary.org/search.json?author=Lucas%2C+Chad&q=Thanks+a+Lot%2C+Universe'


622


2023-11-08 10:19:24,416 - INFO - Author: Kobabe, Maia - Book: Gender Queer: A Memoir - Message: fetching: 'https://covers.openlibrary.org/b/isbn/164700134X-L.jpg'
2023-11-08 10:19:24,416 - INFO - Author: Kobabe, Maia - Book: Gender Queer: A Memoir - Message: fetching: 'https://covers.openlibrary.org/b/isbn/164700134X-L.jpg'
2023-11-08 10:19:24,416 - INFO - Author: Rud, Jeff - Book: Crossover - Message: fetching: 'https://covers.openlibrary.org/b/isbn/164700134X-L.jpg'
2023-11-08 10:19:24,416 - INFO - Author: Rocha, Lucas - Book: Where We Go From Here - Message: fetching: 'https://covers.openlibrary.org/b/isbn/164700134X-L.jpg'
2023-11-08 10:19:24,416 - INFO - Author: Lucas, Chad - Book: Thanks a Lot, Universe - Message: fetching: 'https://covers.openlibrary.org/b/isbn/164700134X-L.jpg'
2023-11-08 10:19:24,753 - WARNING - Author: Kobabe, Maia - Book: Gender Queer: A Memoir - Message: Cover not located for isbn: '164700134X'
2023-11-08 10:19:24,753 - WARNING - Author: Kobabe, Maia - Book

In [59]:
ascii_picture = render_ascii(all_info[0]["cover"], new_width=80)

# for line in ascii_picture[0]:
#     for char in line:
#         print(char, end='')

# print(ascii_picture[0])

final_cover = ""
for line in ascii_picture[0]:
    for char in line:
        final_cover = final_cover + char
    final_cover = final_cover + "\n"

with open("../data/image.txt", "w") as image:
    image.write(final_cover)
print(final_cover)

$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$********=***********************
$$$*********************************************==******************************
$$#================================================**************==******=======
$$#=================*##*=**============================*****************===!!!!!
$$#========================================================***********=!!!!!!!!!
$$#=====================!!==!====!=!!!==!!!!=!!====!!=!===========**==!!!!!!!!!!
$$#=====================!!!!!!!!!!=!!!=!!!!!=!!!=!!!!!!!==============;!!!!!!!!!
$$#===***************=============!!==!===============================!;!!!!!!!!
$$#===$$$$$$$$$$###*====================================================!!;;;;;;
$$#!===#$$$$$#**==****==================================================;;;;;;;;
$$#=====**#*===####*****========================**===***#######*========;;!!!!!!
$$#===============******=================================***###=======!!!!!!!!;;
$$#=======================**